# Projekt Apache Spark

# Wprowadzenie

Wykorzystując ten notatnik jako szablon zrealizuj projekt Apache Spark zgodnie z przydzielonym zestawem. 

Kilka uwag:

* Nie modyfikuj ani nie usuwaj paragrafów *markdown* w tym notatniku, chyba że wynika to jednoznacznie z instrukcji. 
* Istniejące paragrafy zawierające *kod* uzupełnij w razie potrzeby zgodnie z instrukcjami
    - nie usuwaj ich
    - nie usuwaj zawartych w nich instrukcji oraz kodu
    - nie modyfikuj ich, jeśli instrukcje jawnie tego nie nakazują
* Możesz dodawać nowe paragrafy zarówno zawierające kod jak i komentarze dotyczące tego kodu (markdown)

# Treść projektu

Poniżej w paragrafie markdown wstaw tytuł przydzielonego zestawu

# Zestaw 6 – fifa_players

# Działania wstępne 

Utworzenie kontekstów

In [12]:
from pyspark.sql import SparkSession

# Spark session & context
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/10 18:27:15 INFO SparkEnv: Registering MapOutputTracker
24/12/10 18:27:16 INFO SparkEnv: Registering BlockManagerMaster
24/12/10 18:27:16 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/12/10 18:27:16 INFO SparkEnv: Registering OutputCommitCoordinator


In [13]:
sc = spark.sparkContext

W poniższym paragrafie uzupełnij polecenia definiujące poszczególne zmienne. 

Pamiętaj abyś:

* w późniejszym kodzie, dla wszystkich cześci projektu, korzystał z tych zdefiniowanych zmiennych. Wykorzystuj je analogicznie jak parametry
* przed ostateczną rejestracją projektu usunął ich wartości, tak aby nie pozostawiać w notatniku niczego co mogłoby identyfikować Ciebie jako jego autora

In [5]:
# pełna ścieżka do katalogu w zasobniku zawierającego podkatalogi `datasource1` i `datasource4` 
# z danymi źródłowymi
input_dir = "/home/dawiden1/data/zestaw6"

Nie modyfikuj poniższych paragrafów. Wykonaj je i używaj zdefniowanych poniżej zmiennych jak parametrów Twojego programu.

In [6]:
# NIE ZMIENIAĆ
# ścieżki dla danych źródłowych 
datasource1_dir = input_dir + "/datasource1/*"
datasource4_dir = input_dir + "/datasource4/*"

# nazwy i ścieżki dla wyników dla misji głównej 
# część 1 (Spark Core - RDD) 
rdd_result_dir = "/tmp/output1"

# część 2 (Spark SQL - DataFrame)
df_result_table = "output2"

In [7]:
# NIE ZMIENIAĆ
import os
def remove_file(file):
    if os.path.exists(file):
        os.remove(file)

remove_file("metric_functions.py")
remove_file("tools_functions.py")

In [8]:
# NIE ZMIENIAĆ
import requests
r = requests.get("https://jankiewicz.pl/bigdata/metric_functions.py", allow_redirects=True)
open('metric_functions.py', 'wb').write(r.content)
r = requests.get("https://jankiewicz.pl/bigdata/tools_functions.py", allow_redirects=True)
open('tools_functions.py', 'wb').write(r.content)

3322

In [9]:
# NIE ZMIENIAĆ
%run metric_functions.py
%run tools_functions.py

Poniższe paragrafy mają na celu usunąć ewentualne pozostałości poprzednich uruchomień tego lub innych notatników

In [14]:
# NIE ZMIENIAĆ
# usunięcie miejsca docelowego dla część 1 (Spark Core - RDD) 
delete_dir(spark, rdd_result_dir)

Deleted /tmp/output1
Successfully deleted file in HDFS: /tmp/output1


In [16]:
# NIE ZMIENIAĆ
# usunięcie miejsca docelowego dla część 2 (Spark SQL - DataFrame) 
drop_table(spark, df_result_table)

The table output2 does not exist.
Error deleting file file:/spark-warehouse/output2: Command '['hadoop', 'fs', '-rm', '-r', 'file:/spark-warehouse/output2']' returned non-zero exit status 1.


rm: `file:/spark-warehouse/output2': No such file or directory


***Uwaga!***

Uruchom poniższy paragraf i sprawdź czy adres pod którym dostępny Apache Spark Application UI jest poprawny wywołując następny testowy paragraf. 

W razie potrzeby określ samodzielnie poprawny adres pod którym dostępny Apache Spark Application UI

In [29]:
# adres URL, pod którym dostępny Apache Spark Application UI (REST API)
# 
spark_ui_address = extract_host_and_port(spark, "http://localhost:4040")
spark_ui_address

'http://big-data-cluster-m.europe-west4-a.c.braided-grammar-436313-g5.internal:41409'

In [30]:
# testowy paragraf
test_metrics = get_current_metrics(spark_ui_address)

# Część 1 - Spark Core (RDD)

## Misje poboczne

W ponizszych paragrafach wprowadź swoje rozwiązania *misji pobocznych*, o ile **nie** chcesz, aby oceniana była *misja główna*. W przeciwnym przypadku **KONIECZNIE** pozostaw je **puste**.  

## Misja główna 

Poniższy paragraf zapisuje metryki przed uruchomieniem Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [22]:
# NIE ZMIENIAĆ
before_rdd_metrics = get_current_metrics(spark_ui_address)

W poniższych paragrafach wprowadź rozwiązanie *misji głównej* oparte na *RDD API*. 

Pamiętaj o wydajności Twojego przetwarzania, *RDD API* tego wymaga. 

Nie wprowadzaj w poniższych paragrafach żadnego kodu, w przypadku wykorzystania *misji pobocznych*.

In [47]:
data = sc.textFile(datasource1_dir)
parsed_data = data.map(lambda line: line.split(";"))

# Utworzenie klucza (league_id, club_team_id) i wartości 1 dla każdego gracza
players_per_team = parsed_data.map(lambda row: ((row[16], row[17]), 1))

# Zliczenie liczby graczy w każdej drużynie
team_player_counts = players_per_team.reduceByKey(lambda x, y: x + y)

# Filtracja drużyn, które mają co najmniej 11 graczy
teams_with_enough_players = team_player_counts.filter(lambda x: x[1] >= 11)

# Utworzenie klucza league_id i wartości 1 dla każdej drużyny
teams_per_league = teams_with_enough_players.map(lambda x: (x[0][0], 1))

# Zliczenie liczby drużyn w każdej lidze
league_team_counts = teams_per_league.reduceByKey(lambda x, y: x + y)

# Filtracja lig, które mają co najmniej 10 drużyn
leagues_with_enough_teams = league_team_counts.filter(lambda x: x[1] >= 10)

# Pobranie identyfikatorów lig z co najmniej 10 drużynami
valid_league_ids = leagues_with_enough_teams.map(lambda x: x[0]).collect()

# Filtracja piłkarzy grających w ligach spełniających wymagania
filtered_players = parsed_data.filter(lambda row: row[16] in valid_league_ids)


In [57]:
#Narodowosc
players_by_nationality = filtered_players.map(lambda row: (row[25], row)).groupByKey()
nationality_stats = players_by_nationality.mapValues(lambda players: (
    sum(player[10] for player in players),  # sum_value_eur
    sum(player[11] for player in players) / len(players),  # avg_wage_eur
    sum(int(player[12]) for player in players) / len(players),  # avg_age
    len(players)  # count_players
))

# Klub
players_by_club = filtered_players.map(lambda row: (row[18], row)).groupByKey()
club_stats = players_by_club.mapValues(lambda players: (
    sum(player[10] for player in players),  # sum_value_eur
    sum(player[11] for player in players) / len(players),  # avg_wage_eur
    sum(int(player[12]) for player in players) / len(players),  # avg_age
    len(players)  # count_players
))

# Liga
players_by_league = filtered_players.map(lambda row: (row[16], row)).groupByKey()
league_stats = players_by_league.mapValues(lambda players: (
    sum(player[10] for player in players),  # sum_value_eur
    sum(player[11] for player in players) / len(players),  # avg_wage_eur
    sum(int(player[12]) for player in players) / len(players),  # avg_age
    len(players)  # count_players
))

# Łączenie
all_stats = nationality_stats.union(club_stats).union(league_stats)

# Tworzymy RDD z kategorią (nationality, club, league) oraz odpowiednimi statystykami
result = all_stats.map(lambda x: (x[0], {
    'category': 'nationality' if x[0] in nationality_stats else 'club' if x[0] in club_stats else 'league',
    'name': x[0],
    'sum_value_eur': x[1][0],
    'avg_wage_eur': x[1][1],
    'avg_age': x[1][2],
    'count_players': x[1][3]
}))


In [60]:
all_stats.saveAsPickleFile(rdd_result_dir)

24/12/10 19:23:15 WARN TaskSetManager: Lost task 11.0 in stage 60.0 (TID 818) (big-data-cluster-w-0.europe-west4-a.c.braided-grammar-436313-g5.internal executor 16): org.apache.spark.SparkException: Task failed while writing rows
	at org.apache.spark.internal.io.SparkHadoopWriter$.executeTask(SparkHadoopWriter.scala:163)
	at org.apache.spark.internal.io.SparkHadoopWriter$.$anonfun$write$1(SparkHadoopWriter.scala:88)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:93)
	at org.apache.spark.TaskContext.runTaskWithListeners(TaskContext.scala:166)
	at org.apache.spark.scheduler.Task.run(Task.scala:141)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:620)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally(SparkErrorUtils.scala:64)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally$(SparkErrorUtils.scala:61)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:96)
	at org.apache.spark.executor.Executor$TaskRunne

Py4JJavaError: An error occurred while calling o1557.saveAsObjectFile.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:106)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1091)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:410)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopFile$4(PairRDDFunctions.scala:1062)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withS

Poniższy paragraf zapisuje metryki po uruchomieniu Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [ ]:
# NIE ZMIENIAĆ
after_rdd_metrics = get_current_metrics(spark_ui_address)

# Część 2 - Spark SQL (DataFrame)

## Misje poboczne

W ponizszych paragrafach wprowadź swoje rozwiązania *misji pobocznych*, o ile **nie** chcesz, aby oceniana była *misja główna*. W przeciwnym przypadku **KONIECZNIE** pozostaw je **puste**.  

## Misja główna 

Poniższy paragraf zapisuje metryki przed uruchomieniem Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [31]:
# NIE ZMIENIAĆ
before_df_metrics = get_current_metrics(spark_ui_address)

W poniższych paragrafach wprowadź rozwiązanie *misji głównej* swojego projektu oparte o *DataFrame API*. 

Pamiętaj o wydajności Twojego przetwarzania, *DataFrame API* nie jest w stanie wszystkiego "naprawić". 

Nie wprowadzaj w poniższych paragrafach żadnego kodu, w przypadku wykorzystania *misji pobocznych*.

In [32]:
from pyspark.sql import functions as f

spark = SparkSession.builder \
    .appName("Spark Project") \
    .config("spark.hadoop.hadoop.home.dir", "C:/Hadoop") \
    .getOrCreate()

spark.conf.set("spark.sql.debug.maxToStringFields", 10000)

datasource1 = spark.read.csv(
    datasource1_dir,
    sep=";",  # separator
    header=False,  # nagłówki
    inferSchema=True  # wykrycie typów kolumn
)

datasource4 = spark.read.csv(
    datasource4_dir,
    sep=";",  # separator
    header=False,  # nagłówki
    inferSchema=True  # wykrycie typów kolumn
)

leagues_columns = (datasource4.select(f.col("_c0").alias("league_id"),
                                      f.col("_c1").alias("league_name"),
                                      f.col("_c2").alias("league_level")))

selected_columns = (datasource1.select(f.col("_c0").alias("player_id"),
                                       f.col("_c7").alias("player_positions"),
                                       f.col("_c10").alias("value_eur"),
                                       f.col("_c11").alias("wage_eur"),
                                       f.col("_c12").alias("age"),
                                       f.col("_c16").alias("league_id"),
                                       f.col("_c17").alias("club_team_id"),
                                       f.col("_c18").alias("club_name"),
                                       f.col("_c25").alias("nationality_name"))
                    )

In [33]:
agg_players = (selected_columns.groupBy("league_id", "club_team_id")
               .agg(f.count("player_id").alias("total_players"))
               .where(f.col("total_players") >= 11)
               )

agg_leagues = (agg_players.groupBy("league_id")
               .agg(f.count("club_team_id").alias("total_clubs"))
               .where(f.col("total_clubs") >= 10)
               )

reduced_data = (selected_columns
                .join(agg_leagues, on="league_id", how="inner")
                )

top_clubs = (reduced_data.groupBy("club_name")
             .agg(f.sum("value_eur").alias("sum_value_eur"),
                    f.avg("wage_eur").alias("avg_wage_eur"),
                    f.avg("age").alias("avg_age"),
                    f.count("player_id").alias("count_players"),
                    f.collect_set("player_positions").alias("player_positions"))
                .orderBy(f.desc("avg_wage_eur"))
                .select(
    f.lit("club").alias("category"),
    f.col("club_name").alias("name"),
    f.format_number(f.col("sum_value_eur"), 0).alias("sum_value_eur"),
    f.round(f.col("avg_wage_eur"), 3).alias("avg_wage_eur"),
    f.round(f.col("avg_age"), 0).cast("int").alias("avg_age"),
    f.col("count_players").alias("count_players"),
    f.array_distinct(f.col("player_positions")).alias("player_positions"))
                .limit(3))

top_leagues = (reduced_data.join(leagues_columns, 'league_id')
               .groupBy("league_name")
               .agg(f.sum("value_eur").alias("sum_value_eur"),
                    f.avg("wage_eur").alias("avg_wage_eur"),
                    f.avg("age").alias("avg_age"),
                    f.count("player_id").alias("count_players"),
                    f.collect_set("player_positions").alias("player_positions"))
               .orderBy(f.desc("avg_wage_eur"))
               .select(
    f.lit("league").alias("category"),
    f.col("league_name").alias("name"),
    f.format_number(f.col("sum_value_eur"), 0).alias("sum_value_eur"),
    f.round(f.col("avg_wage_eur"), 3).alias("avg_wage_eur"),
    f.round(f.col("avg_age"), 0).cast("int").alias("avg_age"),
    f.col("count_players").alias("count_players"),
    f.array_distinct(f.col("player_positions")).alias("player_positions"))
               .limit(3))

top_nationalities = (reduced_data.groupBy("nationality_name")
               .agg(f.sum("value_eur").alias("sum_value_eur"),
                    f.avg("wage_eur").alias("avg_wage_eur"),
                    f.avg("age").alias("avg_age"),
                    f.count("player_id").alias("count_players"),
                    f.collect_set("player_positions").alias("player_positions"))
               .orderBy(f.desc("avg_wage_eur"))
               .select(
    f.lit("nationality").alias("category"),
    f.col("nationality_name").alias("name"),
    f.format_number(f.col("sum_value_eur"), 0).alias("sum_value_eur"),
    f.round(f.col("avg_wage_eur"), 3).alias("avg_wage_eur"),
    f.round(f.col("avg_age"), 0).cast("int").alias("avg_age"),
    f.col("count_players").alias("count_players"),
    f.col("player_positions")).alias("player_positions")
                .limit(3))

In [34]:
fifa_players = top_clubs.union(top_leagues).union(top_nationalities)
fifa_players.createOrReplaceTempView(df_result_table)

Poniższy paragraf zapisuje metryki po uruchomieniu Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [35]:
# NIE ZMIENIAĆ
after_df_metrics = get_current_metrics(spark_ui_address)

# Analiza wyników i wydajności *misji głównych*

## Część 1 - Spark Core (RDD)

In [ ]:
# Wczytanie wyników z pliku pickle
word_counts = sc.pickleFile(rdd_result_dir)

# Wyświetlenie 50 pierwszych elementów
result_sample = word_counts.take(50)
for item in result_sample:
    print(item)

In [ ]:
subtract_metrics(after_rdd_metrics, before_rdd_metrics)

## Część 2 - Spark SQL (DataFrame)

In [23]:
# Wyświetlenie 50 pierwszych rekordów
fifa_players.show(50)

+-----------+--------------------+--------------+------------+-------+-------------+--------------------+
|   category|                name| sum_value_eur|avg_wage_eur|avg_age|count_players|    player_positions|
+-----------+--------------------+--------------+------------+-------+-------------+--------------------+
|       club|         Real Madrid| 1,050,650,000|  118934.211|     24|           38|[CM, CAM, CM, ST,...|
|       club|     Manchester City| 1,281,910,000|  101384.615|     25|           39|[ST, RW, LW, RW, ...|
|       club|        FC Barcelona| 1,131,875,000|   91571.429|     24|           49|[RW, ST, CAM, CAM...|
|     league|      Premier League|10,481,385,000|   24216.743|     24|         1959|[CB, LB, CDM, RM,...|
|     league|             La Liga| 7,612,290,000|   21643.132|     25|         1143|[LM, RM, LW, CAM,...|
|     league|             Serie A| 7,729,900,000|   19339.702|     26|         1544|[RW, ST, CF, CF, ...|
|nationality|United Arab Emirates|    10,620,0

In [36]:
subtract_metrics(after_df_metrics, before_df_metrics)

{
  "numTasks": 105,
  "numActiveTasks": 0,
  "numCompleteTasks": 105,
  "numFailedTasks": 0,
  "numKilledTasks": 0,
  "numCompletedIndices": 105,
  "executorDeserializeTime": 965,
  "executorDeserializeCpuTime": 788224290,
  "executorRunTime": 2769,
  "executorCpuTime": 2132498495,
  "resultSize": 350918,
  "jvmGcTime": 159,
  "resultSerializationTime": 75,
  "memoryBytesSpilled": 0,
  "diskBytesSpilled": 0,
  "peakExecutionMemory": 0,
  "inputBytes": 34565313,
  "inputRecords": 56939,
  "outputBytes": 0,
  "outputRecords": 0,
  "shuffleRemoteBlocksFetched": 0,
  "shuffleLocalBlocksFetched": 0,
  "shuffleFetchWaitTime": 0,
  "shuffleRemoteBytesRead": 0,
  "shuffleRemoteBytesReadToDisk": 0,
  "shuffleLocalBytesRead": 0,
  "shuffleReadBytes": 0,
  "shuffleReadRecords": 0,
  "shuffleWriteBytes": 0,
  "shuffleWriteTime": 0,
  "shuffleWriteRecords": 0
}
